# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
import os
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
load_dotenv(override=True)
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")

if openrouter_api_key and openrouter_api_key.startswith('-sk-or') and len(openrouter_api_key) > 10:
    print("OpenRouter API key is valid")
else:
    print("OpenRouter API key is invalid")

if google_api_key and google_api_key.startswith('AIza') and len(google_api_key) > 10:
    print("Google API key is valid")
else:
    print("Google API key is invalid")

OpenRouter API key is invalid
Google API key is valid


In [11]:
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"
GOOGLE_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

In [51]:
system_prompt = """
You are an expert technical assistant. You specialize in answering technical questions with clear, accurate, and detailed responses. Provide step-by-step explanations and share your expertise to help users fully understand the concepts behind their inquiries.
"""
user_prompt = """
Can you explain the differences between multiprocessing and multithreading in Python, and when to use each approach?
"""

messages = [
    {"role": "system", "content": system_prompt}
]

In [52]:
gemini_client = OpenAI(api_key=google_api_key, base_url=GOOGLE_BASE_URL)
openrouter_client = OpenAI(api_key=openrouter_api_key, base_url=OPENROUTER_BASE_URL)

def chat_with_selected_model(history, selected_model):
    messages = [{"role": "system", "content": system_prompt}] + history

    # Pick client based on selected model
    if selected_model == "gemini-2.5-flash-lite":
        client = gemini_client
    else:
        client = openrouter_client

    stream = client.chat.completions.create(
        model=selected_model,
        messages=messages,
        stream=True,
    )

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield history + [{"role": "assistant", "content": response}]

    if response == "":
        yield history + [{"role": "assistant", "content": ""}]

def put_message_in_chatbot(message, history):
        return "", history + [{"role":"user", "content":message}]

def set_model(choice):
    print(choice)
    return "gemini-2.5-flash-lite" if choice == "Gemini" else "gpt-4o-mini"

In [ ]:
with gr.Blocks() as ui:
    selected_model = gr.State("gemini-2.5-flash-lite")

    with gr.Row():
        chatbot = gr.Chatbot(height=400, type="messages")

    with gr.Row():
        message = gr.Textbox(label="Chat with our AI Assistant:")

    with gr.Row():
        model_selector = gr.Dropdown(
            choices=["Gemini", "OpenRouter"],
            value="Gemini",
            label="Model"
        )

    model_selector.change(
        set_model,
        inputs=model_selector,
        outputs=selected_model
    )
    message.submit(
        put_message_in_chatbot,
        inputs=[message, chatbot],
        outputs=[message, chatbot]
    ).then(
        chat_with_selected_model,
        inputs=[chatbot, selected_model], 
        outputs=[chatbot]
    )
ui.launch()